<a href="https://colab.research.google.com/github/KNL1979/DS807/blob/main/Part2_RNN_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading libraries

In [ ]:
!pip install wandb -Uq

In [ ]:
import tensorflow as tf

import wandb
from wandb.keras import WandbCallback, WandbMetricsLogger

import pandas as pd
#import requests
#import random
import gzip
import json
import io
import os
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support as score

from sklearn.model_selection import train_test_split

In [ ]:
import psutil

In [ ]:
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
psutil.virtual_memory()[2]

RAM Used (GB): 1.058914304


3.0

# Setting up with Google Drive

In [ ]:
# Mount your google drive where you've saved your assignment folder
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Replace '------' with the path such that "DM890_assignment_1" is your working directory
%cd '/content/gdrive/My Drive/AML_Exam/Problem2'

/content/gdrive/.shortcut-targets-by-id/1osKmyrDLORFSpes1mwrU-W7OXM-to6bK/AML_Exam/Problem2


# Function to sample the data, and data paths

In [ ]:
def dataset_name(file_path):
  dataset_name = os.path.splitext(os.path.basename(file_path))[0].replace('_5.json', '')
  return dataset_name

In [ ]:
def loading_gz(file_path, printing = False, load_all = False, max_load_samples = np.inf, max_samples_per_cat = 2000):
  my_keys = ['overall', 'reviewText', 'summary']
  with gzip.open(file_path, 'rt', encoding='utf-8') as g:
          # Initialize an empty list to store sampled rows
          sampled_data = []
          for i, line in enumerate(g):
              if printing:
                if i % 1000 == 0:
                    print(i)
              data = json.loads(line)
              sampled_data.append({k: data[k] for k in my_keys & data.keys()})
              if i > max_load_samples:
                break
          sampled_data = pd.DataFrame(sampled_data)
          sampled_data.dropna(inplace = True)

          if load_all:
            return sampled_data

          nMax = sampled_data.groupby('overall').count().min()[0] * 3
          #my_max = min(nMax, 2000)
          res = sampled_data.groupby('overall').apply(lambda x: x.sample(n = min(nMax, len(x), max_samples_per_cat))).reset_index(drop = True)

          return res.sample(frac=1)


In [ ]:
# List of file paths for your 29 datasets
file_paths_train = [
    '/content/gdrive/My Drive/AML_Exam/Problem2/data/Arts_Crafts_and_Sewing_5.json.gz',
    '/content/gdrive/My Drive/AML_Exam/Problem2/data/Prime_Pantry_5.json.gz'
]

path_val  = '/content/gdrive/My Drive/AML_Exam/Problem2/data/All_Beauty_5.json.gz'

path_test = '/content/gdrive/My Drive/AML_Exam/Problem2/data/Luxury_Beauty_5.json.gz'

data_path_dict = {dataset_name(k):k for k in file_paths_train}


# Loading data

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Defining Encoding function

In [ ]:
def my_encoder(x_train, x_test = None, MAX_VOCAB_SIZE = 1000, OUTPUT_LENGTH = None):
  encoder = tf.keras.layers.TextVectorization(max_tokens=MAX_VOCAB_SIZE, output_sequence_length = OUTPUT_LENGTH)
  encoder.adapt(x_train)
  # vocab = np.array(encoder.get_vocabulary())

  encoded_x = encoder(x_train)

  if not pd.isnull(x_test).all():
    encoded_x_test = encoder(x_test)
    return encoded_x, encoded_x_test
  else:
    return encoded_x

In [ ]:
data_path_dict

{'Arts_Crafts_and_Sewing': '/content/gdrive/My Drive/AML_Exam/Problem2/data/Arts_Crafts_and_Sewing_5.json.gz',
 'Prime_Pantry': '/content/gdrive/My Drive/AML_Exam/Problem2/data/Prime_Pantry_5.json.gz'}

# Function for builder RNN models

In [ ]:
def model_builder(vocab_size, review_length, embedding_size, RNN_LAYERS: list):
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Embedding(input_dim = vocab_size, output_dim = embedding_size))

    def my_RNN(rnn_dict):
      if rnn_dict['rnn_type'] == 'LSTM':
        layer = tf.keras.layers.LSTM(rnn_dict['width'], return_sequences = rnn_dict['return_seq'])
      elif rnn_dict['rnn_type'] == 'GRU':
        layer = tf.keras.layers.GRU(rnn_dict['width'], return_sequences = rnn_dict['return_seq'])
      if rnn_dict['bi'] == True:
        return tf.keras.layers.Bidirectional(layer, merge_mode = rnn_dict['bi_merge'])
      else:
        return layer

    for layer in RNN_LAYERS:
      model.add(my_RNN(layer))

    model.add(tf.keras.layers.Dense(5, activation='softmax'))

    return model



In [ ]:
RNN_layers_1 = [
    {'rnn_type': 'LSTM',
     'width': 128,
     'return_seq' : True ,
     'bi': True,
     'bi_merge': 'concat'},
    {'rnn_type': 'LSTM',
     'width': 128,
     'return_seq' : True ,
     'bi': True,
     'bi_merge': 'sum'},
    {'rnn_type': 'LSTM',
     'width': 32,
     'return_seq' : False,
     'bi': False
    }]

RNN_layers_2 = [
    {'rnn_type': 'LSTM',
     'width': 128,
     'return_seq' : True ,
     'bi': True,
     'bi_merge': 'sum'},
    {'rnn_type': 'LSTM',
     'width': 128,
     'return_seq' : True ,
     'bi': True,
     'bi_merge': 'concat'},
    {'rnn_type': 'LSTM',
     'width': 32,
     'return_seq' : False,
     'bi': False
    }]


RNN_layers_3 = [
    {'rnn_type': 'LSTM',
     'width': 256,
     'return_seq' : True ,
     'bi': False},
    {'rnn_type': 'LSTM',
     'width': 32,
     'return_seq' : True,
     'bi': False},
    {'rnn_type': 'LSTM',
     'width': 16,
     'return_seq' : False,
     'bi': False
    }]



RNN_layers_4 = [
    {'rnn_type': 'GRU',
     'width': 256,
     'return_seq' : True ,
     'bi': False},
    {'rnn_type': 'GRU',
     'width': 32,
     'return_seq' : True,
     'bi': False},
    {'rnn_type': 'GRU',
     'width': 16,
     'return_seq' : False,
     'bi': False
    }]


RNN_dict = {'model_1': RNN_layers_1, 'model_2': RNN_layers_2, 'model_3': RNN_layers_3, 'model_4': RNN_layers_4}

#RNN_layers = [['LSTM', 200, True , False], ['LSTM', 2, False , False]]
print(model_builder(1500, 100, 128, RNN_dict['model_1']).summary())
print(model_builder(1500, 100, 128, RNN_dict['model_2']).summary())
print(model_builder(1500, 100, 128, RNN_dict['model_3']).summary())
print(model_builder(1500, 100, 128, RNN_dict['model_4']).summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         192000    
                                                                 
 bidirectional (Bidirection  (None, None, 256)         263168    
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, None, 128)         394240    
 onal)                                                           
                                                                 
 lstm_2 (LSTM)               (None, 32)                20608     
                                                                 
 dense (Dense)               (None, 5)                 165       
                                                                 
Total params: 870181 (3.32 MB)
Trainable params: 870181 

# WANDB initialization

In [ ]:
train_name = 'Prime_Pantry'
load_train = loading_gz(data_path_dict[train_name], max_load_samples = 2000000, max_samples_per_cat=50000)
x_train = load_train['reviewText']
y_train = tf.one_hot(load_train['overall']-1,5) #the minus 1 is to adjust for the output of the model being [0,4]

load_val = loading_gz(path_val, load_all=True)
x_val = load_val['reviewText']
y_val = tf.one_hot(load_val['overall']-1,5) #the minus 1 is to adjust for the output of the model being [0,4]

encoded_train, encoded_val  = my_encoder(x_train, x_val, MAX_VOCAB_SIZE = 1500, OUTPUT_LENGTH = 100)

In [ ]:
#wandb.login()
os.environ["WANDB_API_KEY"] = "f1dd10d2eaa5cc353bbe282583f7fc58b9b6c733"
sweep_navn = "ModelTest_4"

In [ ]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'name': sweep_navn,
    'method': 'grid', #grid, random
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
    },
    'parameters': {
        #'training_set_name': {'values': list(data_path_dict.keys())},
        'embedding_size': {'values': [32,64,128,256]},
        #'RNN_layer_width':  {'values': [32, 64, 128]},
        'RNN_layer_model': {'values': list(RNN_dict.keys())},
        #'vocab_size': {'values': [500,1000,1500,2000]},
        #'review_length': {'values': [None, 50, 100, 150, 200]},
        #'epochs': {'values': [20]},
        #'batch_size': {'values': [64]},
        #'dropout': {'values': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]},
        #'learning_rate': {'values': [0.0075]},
        #'optimizer': {'values': ['adam', 'SGD']},
        #'activation': {'values': ['relu']},
    }
}

sweep_id = wandb.sweep(sweep_config, project="RNN_preliminary_tests", entity='coffeegang')

Create sweep with ID: r8db3smm
Sweep URL: https://wandb.ai/coffeegang/RNN_preliminary_tests/sweeps/r8db3smm


## Setting up RNN with gpu

In [ ]:
def train_model():
  config_defaults = {
    #'training_set_name': '',
    'vocab_size': 1500,
    'review_length': 100,
    'RNN_layer_model': 'model_1',
    #'embedding_size': 128,
    #'RNN_layer_width': 64,
    #'RNN_layer_type': 'LSTM',
    #'train_size': int(50000/5),
    'epochs': 50,
    'batch_size': 64,
    'learning_rate': 0.0075,
    'optimizer': 'adam',
    #'validation_split': 0.3,
  }

  with tf.device('/gpu:0'):
    # Initialize a new wandb run
    wandb.init(config=config_defaults)

    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config


    model = model_builder(config.vocab_size, config.review_length, config.embedding_size, RNN_dict[config.RNN_layer_model])

    optimizer = tf.keras.optimizers.Adam(learning_rate = config.learning_rate)

    model.compile(
        loss = "categorical_crossentropy",
        optimizer = optimizer,
        metrics = [tf.keras.metrics.CategoricalAccuracy(),
                   tf.keras.metrics.Precision(),
                   tf.keras.metrics.Recall(),
                   tf.keras.metrics.F1Score()
                   ]+[tf.keras.metrics.Precision(class_id= x, name = f"Precision_{x}") for x in range(5)] +[tf.keras.metrics.Recall(class_id= x, name = f"Recall_{x}") for x in range(5)]
                   ,
        )


    wandb.log({"train_set": train_name, "size_train": len(x_train), 'rnn_layer_model': RNN_dict[config.RNN_layer_model]})

    model.fit(encoded_train, y_train,
              epochs = config.epochs,
              batch_size = config.batch_size,
              validation_data=(encoded_val, y_val),
              callbacks=[WandbCallback(save_model = False),
                          tf.keras.callbacks.EarlyStopping(
                              monitor="loss",
                              #min_delta=0.01,
                              patience=3,
                              #mode="max"
                          )])

In [ ]:
wandb.agent(sweep_id, train_model)

wandb: Agent Starting Run: tieadk91 with config:
wandb: 	RNN_layer_model: model_1
wandb: 	embedding_size: 32
wandb: Currently logged in as: aksel (coffeegang). Use `wandb login --relogin` to force relogin


Epoch 1/50
556/556 [==============================] - 55s 82ms/step - loss: 1.5035 - categorical_accuracy: 0.2923 - precision: 0.1333 - recall: 5.6305e-05 - f1_score: 0.1936 - Precision_0: 0.1333 - Precision_1: 0.0000e+00 - Precision_2: 0.0000e+00 - Precision_3: 0.0000e+00 - Precision_4: 0.0000e+00 - Recall_0: 6.1520e-04 - Recall_1: 0.0000e+00 - Recall_2: 0.0000e+00 - Recall_3: 0.0000e+00 - Recall_4: 0.0000e+00 - val_loss: 1.2928 - val_categorical_accuracy: 0.7165 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.1876 - val_Precision_0: 0.0000e+00 - val_Precision_1: 0.0000e+00 - val_Precision_2: 0.0000e+00 - val_Precision_3: 0.0000e+00 - val_Precision_4: 0.0000e+00 - val_Recall_0: 0.0000e+00 - val_Recall_1: 0.0000e+00 - val_Recall_2: 0.0000e+00 - val_Recall_3: 0.0000e+00 - val_Recall_4: 0.0000e+00
Epoch 2/50
556/556 [==============================] - 23s 41ms/step - loss: 1.4186 - categorical_accuracy: 0.3686 - precision: 0.4889 - recall: 0.0266 - f1_score: 0.2343 

Precision_0,▂▁▁▆▆▆▇▇▇▇▇▇▇████████████
Precision_1,▁▁▁▁▁▁▁▁▁▄▆▇▇▇▇▇▇████████
Precision_2,▁▁▁▅▆▆▇▇▇▇▇▇█████████████
Precision_3,▁▁▆▆▆▆▇▇▇▇▇▇▇▇███████████
Precision_4,▁▅▇▇▇▇▇▇▇▇▇██████████████
Recall_0,▁▁▁▂▄▄▅▅▆▆▆▇▇▇▇▇▇████████
Recall_1,▁▁▁▁▁▁▁▁▁▁▁▂▂▂▄▅▆▆▇▇▇█▇▇▆
Recall_2,▁▁▁▁▃▅▅▆▆▆▇▇▇▇▇▇▇████████
Recall_3,▁▁▁▂▂▃▅▅▆▆▆▇▇▇▇▇▇▇███████
Recall_4,▁▂▆▆▆▇▇▇▇▇▇▇█████████████
categorical_accuracy,▁▂▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇████████


wandb: Agent Starting Run: h8hs2l3m with config:
wandb: 	RNN_layer_model: model_1
wandb: 	embedding_size: 64


Epoch 1/50
556/556 [==============================] - 55s 84ms/step - loss: 1.5171 - categorical_accuracy: 0.2739 - precision: 0.2667 - recall: 4.5044e-04 - f1_score: 0.1741 - Precision_0: 0.0000e+00 - Precision_1: 0.0000e+00 - Precision_2: 0.0000e+00 - Precision_3: 0.2712 - Precision_4: 0.0000e+00 - Recall_0: 0.0000e+00 - Recall_1: 0.0000e+00 - Recall_2: 0.0000e+00 - Recall_3: 0.0016 - Recall_4: 0.0000e+00 - val_loss: 1.2339 - val_categorical_accuracy: 0.8581 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.2084 - val_Precision_0: 0.0000e+00 - val_Precision_1: 0.0000e+00 - val_Precision_2: 0.0000e+00 - val_Precision_3: 0.0000e+00 - val_Precision_4: 0.0000e+00 - val_Recall_0: 0.0000e+00 - val_Recall_1: 0.0000e+00 - val_Recall_2: 0.0000e+00 - val_Recall_3: 0.0000e+00 - val_Recall_4: 0.0000e+00
Epoch 2/50
556/556 [==============================] - 21s 38ms/step - loss: 1.5116 - categorical_accuracy: 0.2720 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.1

...

Precision_0,▁▁▇▇▇▇▇████
Precision_1,▁▁▁▁▁▁▁▁▁▁▁
Precision_2,▁██████████
Precision_3,█▁▂▃▄▄▄▅▄▅▆
Precision_4,▁▅▆█▇█▇████
Recall_0,▁▁▂▃▄▄▅▆▇▇█
Recall_1,▁▁▁▁▁▁▁▁▁▁▁
Recall_2,▁▃▄▅▆▆██▇▆▆
Recall_3,▁▅▆▇▇███▇▇▇
Recall_4,▁▅▆▇▇▇██▇▇▇
categorical_accuracy,▁▅▆▇▇████▇█


wandb: Agent Starting Run: eusbqdba with config:
wandb: 	RNN_layer_model: model_4
wandb: 	embedding_size: 256


Epoch 1/50
556/556 [==============================] - 38s 56ms/step - loss: 1.4649 - categorical_accuracy: 0.3237 - precision: 0.6069 - recall: 0.0331 - f1_score: 0.2154 - Precision_0: 0.0000e+00 - Precision_1: 0.0000e+00 - Precision_2: 0.5900 - Precision_3: 0.4778 - Precision_4: 0.6220 - Recall_0: 0.0000e+00 - Recall_1: 0.0000e+00 - Recall_2: 0.0065 - Recall_3: 0.0088 - Recall_4: 0.1056 - val_loss: 1.3387 - val_categorical_accuracy: 0.2873 - val_precision: 0.8673 - val_recall: 0.1367 - val_f1_score: 0.1312 - val_Precision_0: 0.0000e+00 - val_Precision_1: 0.0000e+00 - val_Precision_2: 0.0000e+00 - val_Precision_3: 0.0515 - val_Precision_4: 0.9821 - val_Recall_0: 0.0000e+00 - val_Recall_1: 0.0000e+00 - val_Recall_2: 0.0000e+00 - val_Recall_3: 0.0151 - val_Recall_4: 0.1539
Epoch 2/50
556/556 [==============================] - 15s 26ms/step - loss: 1.2580 - categorical_accuracy: 0.4442 - precision: 0.6050 - recall: 0.1749 - f1_score: 0.3339 - Precision_0: 0.5833 - Precision_1: 0.0000e+00 

Precision_0,▁▇▇██████
Precision_1,▁▁▁▁▁█▄▁▁
Precision_2,▆▁▄▆▆▇██▆
Precision_3,▁█▇▇███▆▇
Precision_4,▁▄▆▇█▇█▇▅
Recall_0,▁▁▄▆█▇█▇▅
Recall_1,▁▁▁▁▁▅█▁▁
Recall_2,▁▄▆████▇▆
Recall_3,▁▄▅▇███▆▅
Recall_4,▁▆▇████▇▆
categorical_accuracy,▁▆▇█████▆


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


# Final test

In [ ]:
train_name = 'Prime_Pantry'
load_train = loading_gz(data_path_dict[train_name], max_load_samples = 2000000, max_samples_per_cat=50000)
x_train = load_train['reviewText']
y_train = tf.one_hot(load_train['overall']-1,5) #the minus 1 is to adjust for the output of the model being [0,4]

load_val = loading_gz(path_val, load_all=True)
x_val = load_val['reviewText']
y_val = tf.one_hot(load_val['overall']-1,5) #the minus 1 is to adjust for the output of the model being [0,4]

encoded_train, encoded_val  = my_encoder(x_train, x_val, MAX_VOCAB_SIZE = 1500, OUTPUT_LENGTH = 100)

In [ ]:
load_test = loading_gz(path_test, load_all=True)
x_test = load_test['reviewText']
y_test = load_test['overall']-1 #the minus 1 is to adjust for the output of the model being [0,4]
_, encoded_test  = my_encoder(x_train, x_test, MAX_VOCAB_SIZE = 1500, OUTPUT_LENGTH = 100)

In [ ]:
#wandb.login()
os.environ["WANDB_API_KEY"] = "f1dd10d2eaa5cc353bbe282583f7fc58b9b6c733"

In [ ]:
config_defaults = {
    #'training_set_name': '',
    'vocab_size': 1500,
    'review_length': 100,
    'RNN_layer_model': 'model_1',
    'embedding_size': 32,
    #'RNN_layer_width': 64,
    #'RNN_layer_type': 'LSTM',
    #'train_size': int(50000/5),
    'epochs': 100,
    'batch_size': 64,
    'learning_rate': 0.0075,
    'optimizer': 'adam',
    #'validation_split': 0.3,
  }

In [ ]:
wandb.init(config=config_defaults, project="RNN_preliminary_tests", entity='coffeegang', name='final_1')

# Config is a variable that holds and saves hyperparameters and inputs
config = wandb.config


model = model_builder(config.vocab_size, config.review_length, config.embedding_size, RNN_dict[config.RNN_layer_model])

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate = config.learning_rate)

model.compile(
    loss = "categorical_crossentropy",
    optimizer = optimizer,
    metrics = [tf.keras.metrics.CategoricalAccuracy(),
                tf.keras.metrics.Precision(),
                tf.keras.metrics.Recall(),
                tf.keras.metrics.F1Score()]
    )


wandb.log({"train_set": train_name, "size_train": len(x_train), 'rnn_layer_model': RNN_dict[config.RNN_layer_model]})

model.fit(encoded_train, y_train,
          epochs = config.epochs,
          batch_size = config.batch_size,
          validation_data = [encoded_val, y_val],
          callbacks=[WandbCallback(save_model = False)])

In [ ]:
pred = model.predict(encoded_test)

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y_test, np.argmax(pred,axis=1))).plot()

In [ ]:
print(classification_report(y_test, np.argmax(pred,axis=1)))

In [ ]:
y_val_sparse = load_val['overall']-1
pred_val = model.predict(encoded_val)
ConfusionMatrixDisplay(confusion_matrix(y_val_sparse, np.argmax(pred_val,axis=1))).plot()
print(classification_report(y_val_sparse, np.argmax(pred_val,axis=1)))